# El modelo de equilibrio general estático

## Sector público y análisis de la incidencia de impuestos

Ahora vamos a introducir la actividad del gobierno en nuestro modelo. El gobierno va a proveer un nivel exógeno de bien público $G$ y va a recaudar impuestos sobre el consumo y la renta para financiarlo. Vamos a asumir que el bien público es equivalente al Bien 1. En esta economía, tenemos que distinguir entre los precios de consumo y los precios de producción, así como entre los precios brutos y netos de los factores
$$p_i = q_i(1+\tau_i), \ w^n = w(1-\tau_w), \ r^n = r(1-\tau_r),$$
donde $\tau_i$ denota los impuestos sobre el consumo, y $\tau_w, \tau_r$ los impuestos sobre los salarios y los intereses, respectivamente. 

Ahora la restricción presupuestaria de los consumidores está definida por
$$p_1 X_1 + p_2 X_2 + w^n \ell = w^n \bar{T} + r^n \bar{K} := \bar{Y}^n,$$
que da lugar a las nuevas funciones de demanda de consumo y de ocio
$$X_1 = \frac{\alpha_1}{p_1}\bar{Y}^n, \ X_2 = \frac{\alpha_2}{p_2}\bar{Y}^n, \ \ell = \frac{1-\alpha_1-\alpha_2}{w^n}\bar{Y}^n.$$

En el lado del productor, las funciones de demanda de factores del modelo no cambian. Como el gobierno demanda cierta candidad del Bien 1, la condición de equilibrio del mercado de este bien ahora es $X_1 + G = Y_1$. La condición de equilibrio del mercado del Bien 2 no va a cambiar, pero debemos tener en cuenta las diferencias entre los precios de producción y de consumo. En consecuencia, las condiciones de equilibrio de los mercados de los dos bienes son
$$\frac{\alpha_1 \bar{Y}^n}{p_1} + G - \left(\frac{\beta_1}{w}\right)^{\beta_1} \left(\frac{1-\beta_1}{r}\right)^{1-\beta_1} q_1 \left(\frac{\alpha_1 \bar{Y}^n}{p_1} + G\right) = 0,$$
$$1-\left(\frac{\beta_2}{w}\right)^{\beta_2} \left(\frac{1-\beta_2}{r}\right)^{1-\beta_2} q_2 = 0.$$

La condición de equilibrio del mercado de trabajo:
$$\frac{\beta_1}{w}q_1 \left(\frac{\alpha_1 \bar{Y}^n}{p_1} + G\right) + \frac{\beta_2}{w}q_2\frac{\alpha_2 \bar{Y}^n}{p_2} + \frac{1-\alpha_1-\alpha_2}{w^n}\bar{Y}^n - \bar{T} = 0.$$

Finalmente, en equilibrio, los gastos del gobierno tienen que ser financiados con impuestos:
$$q_1 G = \sum_{i=1}^2 \tau_i q_i X_i + \tau_w w (\bar{T} - \ell) + \tau_r r \bar{K},$$
que puede ser reescrita de la siguiente manera:
$$q_1 G - \sum_{i=1}^2 \frac{\tau_i}{1+\tau_i}\alpha_i \bar{Y}^n - \tau_w w \left( \bar{T} - \frac{1-\alpha_1-\alpha_2}{w^n}\bar{Y}^n \right) - \tau_r r \bar{K},$$
donde uno de los impuestos es endógeno y los otros tres son exógenos. Dado $G$ y los impuestos que especificamos de manera exógena, es posible resolver el sistema de cuatro ecuaciones (tres mercados y la restricción presupuestaria del gobierno) y cuatro incógnitas ($p_2$, $w$, $r$, y el impuesto endógeno). 

In [4]:
#===============================================================================
# PROGRAMA: Solución de mercado al modelo de equilibrio general estático con
# sector público.
#
# AUTHOR: Manuel V. Montesinos (El Economista Camuflado).
#
# REFERENCIA: Fehr, H., y Kindermann F. (2018), 
# "Introduction to Computational Economics using Fortran",
# Oxford University Press.
#
# FECHA: 5 de enero de 2025.
#===============================================================================

# Importar librerías
import numpy as np
from scipy.optimize import fsolve

print("-----------------------------------------------------------------------")
print(" ")
print("Solución de mercado al modelo de equilibrio general estático con")
print("sector público.")
print(" ")
print("-----------------------------------------------------------------------")

# Cantidad disponible de capital
Kbar = 10

# Cantidad de tiempo disponible
Tbar = 30

# Parámetros de las preferencias Cobb-Douglas y la función de producción
alpha = np.array([0.3, 0.4])
beta = np.array([0.3, 0.6])

# Nivel de bien público
G = 3

# Impuesto sobre las rentas del trabajo
tauw = 0

# Impuesto sobre las rentas del capital
taur = 0

# Impuestos sobre el consumo
tauc = np.array([0, 0])

# Función para determinar el equilibrio de mercado
def markets(x):

    # Precios de producción e impuestos
    q = np.array([1, x[0]])
    w = x[1]
    r = x[2]

    # Impuestos
    taur = x[3]
    #tauw = -x[3]
    #tauc = np.array([x[3], 0])

    # Calcula los precios de consumo y la renta total
    p = q*(1 + tauc)
    wn = w*(1 - tauw)
    rn = r*(1 - taur)
    Ybarn = wn*Tbar + rn*Kbar

    # Ecuaciones de equilibrio
    ms1 = alpha[0]*Ybarn/p[0] + G - (beta[0]/w)**beta[0] * \
        ((1-beta[0])/r)**(1-beta[0]) * q[0]*(alpha[0]*Ybarn/p[0] + G)
    ms2 = 1/p[1] - (beta[1]/w)**beta[1] * \
            ((1-beta[1])/r)**(1-beta[1])*q[1]/p[1]
    ms3 = beta[0]/w * q[0]*(alpha[0]*Ybarn/p[0]+G) + \
            beta[1]/w * q[1]*alpha[1]*Ybarn/p[1] + \
            (1-alpha[0]-alpha[1])*Ybarn/wn - Tbar
    ms4 = q[0]*G - tauc[0]/(1+tauc[0])*alpha[0]*Ybarn - \
            tauc[1]/(1+tauc[1])*alpha[1]*Ybarn - \
            tauw*w*(Tbar-(1-alpha[0]-alpha[1])/wn * Ybarn) - taur*r*Kbar
    
    return [ms1, ms2, ms3, ms4]

# Valores iniciales para los precios y los impuestos en equilibrio
x0 = np.array([0.5, 0.5, 0.5, 0.5])

# Obtener el equilibrio de mercado
equil = fsolve(markets, x0)
q = [1, equil[0]]
w = equil[1]
r = equil[2]

# Impuestos en equilibrio
taur = equil[3]
#tauw = equil[3]
#tauc = np.array([equil[3], 0])

# Precios de consumo y renta total
p = q*(1 + tauc)
wn = w*(1 - tauw)
rn = r*(1 - taur)

# Calcular otras variables
Ybarn = wn*Tbar + rn*Kbar
Xd = alpha*Ybarn/p
Y = [Xd[0] + G, Xd[1]]
ell = (1-alpha[0]-alpha[1])*Ybarn/wn
L = beta*q*Y/w
K = (1-beta)*q*Y/r
U = Xd[0]**alpha[0]*Xd[1]**alpha[1]*ell**(1-alpha[0]-alpha[1])

# Mostrar resultados
print(" ")
print("Mercado del bien 1: ")
print("X1 =", Xd[0], " G =", G, " Y1 =", Y[0])
print("q1 =", q[0], " p1 =", p[0], " tc1=", tauc[0])
print(" ")
print("Mercado del bien 2: ")
print("X2 =", Xd[1], " G =", 0, " Y2 =", Y[1])
print("q2 =", q[1], " p2 =", p[1], " tc2=", tauc[1])
print(" ")
print("Mercado de trabajo: ")
print("L1 =", L[0], " L2 =", L[1], " T-F =", Tbar-ell)
print("w =", w, " wn =", wn, " tw =", tauw)
print(" ")
print("Mercado de capital: ")
print("K1 =", K[0], " K2 =", K[1], " K =", Kbar)
print(" r =", r, " rn =", rn, " tr =", taur)
print(" ")
print("Gobierno: ")
print("tc1 =", tauc[0]*q[0]*Xd[0], " tc2 =", tauc[1]*q[1]*Xd[1], \
      " tw =", tauw*w*(Tbar-ell), " tr =", taur*r*Kbar, " G =", q[0]*G)
print(" ")
print("Bienestar: ")
print("U =", U)
print(" ")
print("-----------------------------------------------------------------------")



-----------------------------------------------------------------------
 
Solución de mercado al modelo de equilibrio general estático con
sector público.
 
-----------------------------------------------------------------------
 
Mercado del bien 1: 
X1 = 3.9313623017017227  G = 3  Y1 = 6.931362301701723
q1 = 1  p1 = 1.0  tc1= 0
 
Mercado del bien 2: 
X2 = 6.305054808519217  G = 0  Y2 = 6.305054808519217
q2 = 0.8313673015476988  p2 = 0.8313673015476988  tc2= 0
 
Mercado de trabajo: 
L1 = 6.813369256006686  L2 = 10.305169219644648  T-F = 17.118538475444193
w = 0.3051953611170133  wn = 0.3051953611170133  tw = 0
 
Mercado de capital: 
K1 = 6.982554227536699  K2 = 3.0174457723760773  K = 10
 r = 0.69486801721594  rn = 0.3948680172162011  tr = 0.43173666447006687
 
Gobierno: 
tc1 = 0.0  tc2 = 0.0  tw = 0.0  tr = 2.9999999999973896  G = 3
 
Bienestar: 
U = 6.779973701645119
 
-----------------------------------------------------------------------
